In [1]:
import json
import torch
import pandas as pd

from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_int8_training
from contextlib import nullcontext
from transformers import (default_data_collator, Trainer, TrainingArguments,
                          TrainerCallback, AutoTokenizer, AutoModelForCausalLM,
                          BitsAndBytesConfig)

from transformers.integrations import WandbCallback

from utils.dataset import CombineDataset, template

# setting up wandb
%env WANDB_PROJECT=disco-limbic-dialogue

env: WANDB_PROJECT=disco-limbic-dialogue


# Params

In [2]:
# model
model_id = 'microsoft/phi-2'
model_type = 'phi'

# data settings
dataset_train_path = 'data/dataset/v1/train.json'
dataset_test_path = 'data/dataset/v1/test.json'
max_data_length = 384

# lora settings
lora_r = 32
lora_alpha = 16
lora_dropout = 0.05
target_modules = ['Wqkv', 'out_proj']

# train settings 
device = 'cuda'
lr = 3e-4
num_train_epochs = 5

gradient_accumulation_steps = 32
per_device_train_bs = 1
per_device_eval_bs = 1

log_steps = 10
eval_steps = 30


output_dir = f'lora/disco-limbic-dialogue-phi2-eos'

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    flash_attn=True,
    flash_rotary=True,
    fused_dense=True,
    trust_remote_code=True,
    device_map={'': 0},
    quantization_config=bnb_config
)


tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# load data

In [4]:
with open(dataset_train_path, 'r') as f:
    raw_data = json.load(f)

with open(dataset_test_path, 'r') as f:
    raw_data_test = json.load(f)

## filter
data_train = []
for i in raw_data:
    if len(tokenizer(template(i, model_type))['input_ids']) < max_data_length:
        data_train.append(i)
print(len(raw_data), len(data_train))

data_test = []
for i in raw_data_test:
    if len(tokenizer(template(i, model_type))['input_ids']) < max_data_length:
        data_test.append(i)
print(len(raw_data_test), len(data_test))

#data_train, data_test = torch.utils.data.random_split(data, lengths=[0.95, 0.05])
train_dataset = CombineDataset(data_train, tokenizer,
                               max_words=max_data_length,
                               model_type=model_type)
test_dataset = CombineDataset(data_test, tokenizer,
                              max_words=max_data_length,
                              model_type=model_type)
len(train_dataset), len(test_dataset)

4768 4755
118 118


(4755, 118)

In [5]:
model.train()

def create_peft_config(model):
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules = target_modules
    )

    model = prepare_model_for_int8_training(model, use_gradient_checkpointing=False)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)

trainable params: 15,728,640 || all params: 2,790,783,096 || trainable%: 0.5635923487763593


/home/xx/miniconda3/envs/disco/lib/python3.10/site-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [6]:
enable_profiler = False
config = {
    'lora_config': lora_config,
    'learning_rate': lr,
    'num_train_epochs': num_train_epochs,
    'gradient_accumulation_steps': gradient_accumulation_steps,
    'per_device_train_batch_size': per_device_train_bs,
    'gradient_checkpointing': False,
}

profiler = nullcontext()


In [7]:
def decode_predictions(tokenizer, predictions):
    prediction_text = tokenizer.batch_decode(predictions.predictions.argmax(axis=-1))
    return {"predictions": prediction_text} # "labels": labels, 


class WandbPredictionProgressCallback(WandbCallback):
    def __init__(self, trainer, tokenizer, val_dataset, num_samples=100, freq=2):
        super().__init__()
        self.trainer = trainer
        self.tokenizer = tokenizer
        self.sample_dataset = [val_dataset[i] for i in range(num_samples)]
        self.freq = freq

    def on_evaluate(self, args, state, control, **kwargs):
        super().on_evaluate(args, state, control, **kwargs)
        if state.global_step % self.freq == 0:
            predictions = self.trainer.predict(self.sample_dataset)
            predictions = decode_predictions(self.tokenizer, predictions)
            predictions_df = pd.DataFrame(predictions)
            predictions_df["epoch"] = state.epoch
            records_table = self._wandb.Table(dataframe=predictions_df)
            self._wandb.log({"sample_predictions": records_table})

In [8]:
print(output_dir)
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    #bf16=True,  # Use BF16 if available
    ## eval strat
    do_eval=True,
    evaluation_strategy='steps',
    eval_steps=eval_steps,
    per_device_eval_batch_size=per_device_eval_bs,
    ## logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=log_steps,
    ## wandb
    report_to="wandb",
    run_name=output_dir.split('/')[-1],
    ## other
    save_strategy="no",
    #optim="adamw_torch_fused",
    max_steps=-1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    # Create Trainer instance``
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=default_data_collator,
        callbacks=[],
    )
    # Instantiate the WandbPredictionProgressCallback
    progress_callback = WandbPredictionProgressCallback(
        trainer=trainer,
        tokenizer=tokenizer,
        val_dataset=test_dataset,
        num_samples=16,
        freq=30,
    )

    # Add the callback to the trainer
    trainer.add_callback(progress_callback)
    
    trainer.train()

lora/disco-limbic-dialogue-phi2-eos


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xxond. Use `wandb login --relogin` to force relogin


  0%|          | 0/740 [00:00<?, ?it/s]

{'loss': 3.2236, 'learning_rate': 0.0002959459459459459, 'epoch': 0.07}
{'loss': 2.6829, 'learning_rate': 0.0002918918918918919, 'epoch': 0.13}
{'loss': 2.4297, 'learning_rate': 0.0002878378378378378, 'epoch': 0.2}


  0%|          | 0/118 [00:00<?, ?it/s]

...

{'loss': 1.5085, 'learning_rate': 1.6216216216216215e-05, 'epoch': 4.71}
{'loss': 1.4209, 'learning_rate': 1.2162162162162162e-05, 'epoch': 4.78}
{'loss': 1.375, 'learning_rate': 8.108108108108107e-06, 'epoch': 4.85}


  0%|          | 0/118 [00:00<?, ?it/s]

{'eval_loss': 1.6428526639938354, 'eval_runtime': 71.2188, 'eval_samples_per_second': 1.657, 'eval_steps_per_second': 1.657, 'epoch': 4.85}


  0%|          | 0/16 [00:00<?, ?it/s]

{'loss': 1.4029, 'learning_rate': 4.054054054054054e-06, 'epoch': 4.91}
{'loss': 1.4102, 'learning_rate': 0.0, 'epoch': 4.98}
{'train_runtime': 31989.8005, 'train_samples_per_second': 0.743, 'train_steps_per_second': 0.023, 'train_loss': 1.7601289143433443, 'epoch': 4.98}


In [9]:
torch.cuda.current_device()

0

In [10]:
model.save_pretrained(output_dir)

In [11]:

dialog = [
    #"[Electrochemistry]: Whoa! In your hand: *pyrholidon* -- the double rainbow of synthetic hallucinogens. Rare and gritty, a product of the age of atomic power.",
    #"Look at the little puck of liquid.",
    #"[Electrochemistry]: What a funny little cap! Don't let the *scary* medical warnings throw you off. It's an inadequate antidote to radiation poisoning, but a *potent* antidote to *boredom*.",
    #"Hmm... open the cap.",
    #'Look around',
    #'"How can I take shit without taking off my sweater?"',
    'who am i?</s>'
]



query = template(dialog) + ' [|Assistant|] '
model_inputs = tokenizer(query, return_tensors="pt", add_special_tokens=False).input_ids.to('cuda')
generated_ids = model.generate(input_ids=model_inputs, max_new_tokens=64,
                               do_sample=True,
                               #pad_token_id=tokenizer.eos_token_id,
                               temperature=0.7,
                               repetition_penalty=1.15)
#outputs = model(input_ids=input_ids),# max_length=cut_len, min_length=8, top_p=0.9, do_sample=True)
output = tokenizer.decode(generated_ids[0], skip_special_tokens=False)
print(output)

TypeError: template() missing 1 required positional argument: 'model_type'